In [28]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from helper_functions import elementlines, nearestlines

In [29]:
%matplotlib qt

### Variables that you need to change

- path
    - path to the data
- file
    - file name
- elements
    - list of elements to be analyzed
- lines_of_interest
    - lines of interest, will be included in the output
- info_on_all_lines
    - boolean, if true, the output will contain all lines, not only the lines of interest
- zero_peak
    - boolean, if true, the zero peak will be sliced out
- zero_peak_end_index
    - index of the last element of the zero peak, which will be sliced out
- line_ratio_list
    - list of list of lines which will be given a ratio, eg. K to L
    - [['Ni_Ka', 'Ni_La'], ['Ni_Ka', 'Ni_Kb']]
- fwtm_to_fwhm_lines
    - lines of interest for the fwtm to fwhm ratio, should not be overlapping with other peaks
    - the fwhm and fwtm here is taken from the raw spectrum, since the fitted gaussians have perfect shapes
    - then if a overlapping peak is used, the ratio will be skewed

In [30]:
#### SU9000 #####
path = 'data/Skomedal_2022-03-23_EDS_SU9000'
file = 'Spectrum 03.emsa'
elements = ['Al', 'Au', 'C', 'Cu', 'F', 'Fe', 'Mg', 'Mo', 'Ni', 'O', 'Si', 'Sn']  # from Mari, pluss Mg and Sn
lines_of_interest = ['Al_Ka', 'C_Ka', 'Cu_Ka', 'Mo_La', 'Ni_Ka', 'Ni_La', 'O_Ka', 'Si_Ka']
info_on_all_lines = False
line_ratio_list = [['Ni_Ka', 'Ni_La']]
fwtm_to_fwhm_lines = ['Ni_Ka']

# ##### GaAs 30 kV APREO #####
path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
# file = 'GaAs_30kV.emsa'
# file = 'GaAs_15kV.emsa'
file = 'GaAs_10kV.emsa'

elements = ['Ga', 'As', 'O', 'C', 'Si']
lines_of_interest = ['Ga_Ka', 'As_Ka', 'O_Ka', 'C_Ka', 'Si_Ka', 'Ga_La', 'As_La', 'Ga_Kb', 'As_Kb', 'Ga_Ll']
info_on_all_lines = False
line_ratio_list = [['As_Ka', 'As_La'], ['Ga_Ka', 'Ga_La']]
fwtm_to_fwhm_lines = ['Ga_Ka']


# ##### Mo APREO #####
# path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
# file = 'Mo_30kV.emsa'
# elements = ['Mo', 'O']
# lines_of_interest = ['Mo_Ka', 'O_Ka', 'Mo_La', 'Mo_Kb', 'Mo_Ll', 'Mo_Lb1', 'Mo_Lg3']
# info_on_all_lines = False
# line_ratio_list = [['Mo_Ka', 'Mo_La']]
# fwtm_to_fwhm_lines = ['Mo_Ka', 'Mo_La']

zero_peak = True 
zero_peak_end_index = 33

In [31]:
# function arguments that you can change, eg. if a step take too long

# the model
model_background_order = 6


## The output parameters that will be calculated are:

...

In [32]:
# output parameters
scale = np.nan  # float
offset = np.nan  # float 
energy_resolution = np.nan  # float
total_counts = np.nan  # float
background_counts = np.nan  # float
dh_limit = np.nan # float
lines_info = dict() # dict of dicts as {line_name1: {true_energy: ..., calib_energy: ..., area: ..., max_counts: ..., sigma: ..., fwhm: ...}, line_name2: {...}}
line_ratios = dict()  # dict as {line1a/line1b: ratio1, line2a/line2b: ratio2, ...}
fwtm_to_fwhm = dict()  # dict as {line1: fwtm_to_fwhm1, line2: fwtm_to_fwhm2, ...}

## Loading the data and setting some parameters

In [33]:
s = hs.load(path + '/' + file, signal_type='EDS_TEM')  # have to pretent it's a TEM signal
s.add_elements(elements)

if zero_peak:
    s = s.isig[zero_peak_end_index:]

Vacc = s.metadata.Acquisition_instrument.TEM.beam_energy
x_max = s.axes_manager[0].high_value  # highest x-axis value in keV, used in Duane-Hunt
x = s.axes_manager[0].axis  # x-axis in keV

In [34]:
plt.rcParams['font.size'] = '16'
plt.rcParams["figure.figsize"] = (15,8)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'monospace'

In [35]:
# plot the spectrum

# s.plot(xray_lines=True)

In [36]:
# these are temporary arrays used to show the effect of the calibrations
# temporary because thay are not used in the final output

scale_list = [s.axes_manager[0].scale]
offset_list = [s.axes_manager[0].offset]
energy_res_list = [s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa]

In [52]:
# # Duane-Hunt method to find the real E_0
def calculate_duane_hunt(buffer_start=2, buffer_end=0.1, xaxis_plot_buffer=0.5, dh_plot=False):
    if Vacc > x_max:
        print(f'Vacc={Vacc} > x_max={x_max}, Duane-Hunt not possible')
        return np.nan
    else:
        s_dh = s.deepcopy() 
        # making the lin fit of the background right before Vacc
        dh_start = Vacc-buffer_start
        dh_end = Vacc-buffer_end
        s_end = s_dh.isig[dh_start:dh_end] # slice with keV
        m_end = s_end.create_model(auto_background=False)
        m_end.add_polynomial_background(order=1)
        m_end.fit()

        x_s_end = s_dh.isig[dh_start-xaxis_plot_buffer:dh_end+xaxis_plot_buffer].axes_manager[0].axis
        dh_bg_zero_index = np.argmin(np.abs(m_end[-1].function(x_s_end) * s_dh.axes_manager[0].scale))

        dh_limit = x_s_end[dh_bg_zero_index]
        print(f'dh_bg_zero_index: {dh_bg_zero_index}')
        print(f'Duane-Hunt limit: {dh_limit:.3f} keV')

        if dh_plot:
            plt.plot(x, s_dh.data, label='spectrum', marker='o')
            plt.plot(x_s_end, m_end[-1].function(x_s_end)* s_dh.axes_manager[0].scale, label='bg lin. fit')
            plt.plot(s_end.axes_manager[0].axis, s_end.data, marker='o', label=f'points in bg lin. fit')
            plt.axhline(0, color='k', linestyle='--')
            plt.axvline(dh_limit, color='r', linestyle='--', label=f'Duane-Hunt: {dh_limit:.3f} keV')
            plt.axvline(Vacc, color='y', linestyle='--', label=f'Vacc from instrument: {Vacc} keV', alpha=0.6)
            plot_buffer = 1.1
            plt.title(f'{file}\nlin fit range kV: [{dh_start:.2f}, {dh_end:.2f}]')
            plt.ylim(-5, m_end[-1].function(x_s_end).max() * s_dh.axes_manager[0].scale*plot_buffer)
            plt.xlim(Vacc-buffer_start*plot_buffer, dh_limit + buffer_end)
            plt.legend()
            plt.show()
        
        return dh_limit

In [53]:
dh_limit = calculate_duane_hunt(dh_plot=True)

dh_bg_zero_index: 262
Duane-Hunt limit: 10.120 keV


In [54]:
# Using Duane-Hunt to slice the spectrum
def use_dh_to_slice_spectrum(dh_limit=dh_limit, s=s, plot=False):
    if np.isnan(dh_limit):
        print('No Duane-Hunt limit found, not slicing the spectrum')
    else:
        s = s.isig[:dh_limit]
        print(f'Spectrum sliced at {dh_limit:.2f} keV')
        if plot:
            s.plot(xray_lines=True)
    return s

In [55]:
s = use_dh_to_slice_spectrum(dh_limit, plot=True)
x = s.axes_manager[0].axis  # x-axis in keV, after slicing

Spectrum sliced at 10.12 keV


In [56]:
# creating a model
m = s.create_model(auto_background=False)

# fitting the data
m.add_polynomial_background(order=model_background_order) 
m.fit_background()
m.fit(bounded=True)
m.plot(True)

## Calibration (scale, offset and resolution)

In [77]:
#  Functions for calibrating the energy axis

def calibrate_axis(rounds=2, xray_lines='all_alpha'):
    # calibration of the energy axis, i.e. scale and offset
    print('Calibrating energy axis (with many elements it can take multiple minutes)')
           
    for i in range(rounds):
        print(f'Calibrating offset and scale, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='offset', xray_lines=xray_lines)
        offset_list.append(s.axes_manager[0].offset)
        m.calibrate_energy_axis(calibrate='scale', xray_lines=xray_lines)
        scale_list.append(s.axes_manager[0].scale)

    print(f'Scale: {scale_list[-1]:.6f} eV/px \nOffset: {offset_list[-1]:.6f} keV')
    scale = scale_list[-1]
    offset = offset_list[-1]


def calibrate_resolution(rounds=2, xray_lines='all_alpha'):
    # calibration of the energy resolution
    for i in range(rounds):
        print(f'Calibrating energy resolution, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
        energy_res_list.append(s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa)

    print(f'Calibrated energy resolution: {energy_res_list[-1]:.3f} eV')
    energy_resolution = energy_res_list[-1]


def print_calibration_info():
    # make pretty print of calibration info
    infos = [' ', 'Scale [eV/channel]', 'Offset [keV]', 'E-res [eV]']
    row1 = ['Current', f'{scale_list[-1]:.6f}', f'{offset_list[-1]:.6f}', f'{energy_res_list[-1]:.3f}']
    row2 = ['Original', f'{scale_list[0]:.6f}', f'{offset_list[0]:.6f}', f'{energy_res_list[0]:.3f}']
    row3 = ['Δ original', f'{(scale_list[-1] - scale_list[0])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[0])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[0])/energy_res_list[0]*100:.3f} %']
    row4 = ['Δ last step', f'{(scale_list[-1] - scale_list[-2])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[-2])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[-2])/energy_res_list[-2]*100:.3f} %']

    for i in range(len(infos)):
        print(f'{infos[i]:<20}{row1[i]:<15}{row2[i]:<15}{row3[i]:<15}{row4[i]:<15}')
    


In [78]:
calibrate_axis()
calibrate_resolution()

Calibrating energy axis (with many elements it can take multiple minutes)
Calibrating offset and scale, round 1 of 2
Calibrating offset and scale, round 2 of 2
Scale: 0.009997 eV/px 
Offset: 0.134898 keV
Calibrating energy resolution, round 1 of 2


Calibrating energy resolution, round 2 of 2
Calibrated energy resolution: 129.920 eV


In [79]:
print_calibration_info()

                    Current        Original       Δ original     Δ last step    
Scale [eV/channel]  0.009997       0.010000       -0.032 %       -0.001 %       
Offset [keV]        0.134898       0.130000       3.635 %        0.122 %        
E-res [eV]          129.920        130.000        -0.061 %       -0.001 %       


## Peak positions fitting

In [80]:
# helper functions


def energy_index(energy):
    # given an energy in keV, return the index of that energy in the spectrum
    return np.abs(x - energy).argmin()


def energy_counts(energy):
    # given an energy in keV, return the counts at that energy
    energy_index = energy_index(energy)
    return s.data[energy_index]


def range_counts(start, stop):
    # given a range in keV, return the counts in that range
    return s.data[energy_index(start), energy_index(stop)].sum()


def max_counts(energy, buffer=10):
    # get the highest counts in +- 10 channels around a given energy
    i = np.argmin(np.abs(x - energy))
    return s.data[i-buffer:i+buffer].max()
     

def gaussian_counts(line):
    # give the sum of counts from a gaussian fit of a line
    return (m[line].function(x) * s.axes_manager[0].scale).sum()

def gaussian_max_counts(line):
    # give the max counts from a gaussian fit of a line
    return m[line].function(x).max() * s.axes_manager[0].scale


def bg_point(energy):
    # give the background counts at a certain energy
    return m[-1].function(energy) * s.axes_manager[0].scale


def bg_range(range_array):
    # give the background sum counts in range, where range is an array of energies
    return (m[-1].function(range_array) * s.axes_manager[0].scale).sum()

    
def kev_to_channels(energy):
    return np.argmin(np.abs(x - energy))


def theoretical_energy(line):
    # returns the theoretical energy of a given line, e.g. 'Ga_Ka'
    element = line.split('_')[0]
    line_name = line.split('_')[1]
    return hs.material.elements[element]['Atomic_properties']['Xray_lines'][line_name]['energy (keV)']


In [81]:
# functions for calibrations of lines

def calibrate_lines(rounds=2, xray_lines='all_alpha'):
    # calibrating the lines using HyperSpy
    # calibrate energy rounds times, and width and sub_weight only once
    for i in range(rounds):
        print(f'Calibrating peak positions, round {i+1} of {rounds}')
        m.calibrate_xray_lines(calibrate='energy', xray_lines=xray_lines, kind='single') # use kind='multi' for better results?
        if i == 0:
            m.calibrate_xray_lines(calibrate='width', xray_lines=xray_lines, kind='single')
            m.calibrate_xray_lines(calibrate='sub_weight', xray_lines=xray_lines, kind='single')


def make_lines_info(all_lines=info_on_all_lines):
    # make dict with lines info
    lines_info = {}
    for i in range(len(m) - 1): # last component is the background
        if (all_lines == True) or (m[i].name in lines_of_interest):
            lines_info[m[i].name] = {
                'true_energy': theoretical_energy(m[i].name),
                'calib_energy': m[i].centre.value,
                'area': gaussian_counts(i),
                'max_counts': gaussian_max_counts(i),
                'sigma': m[i].sigma.value,
                'fwhm': m[i].fwhm * 1000
            }
    return lines_info


def print_lines_info(give_return=False):
    # print line info
    headers = ['Line', 'True E [keV]', 'Calib. E [keV]', 'Area [counts]', 'Max (fit)', 'Sigma [keV]', 'FWHM [eV]']
    table = ''
    for i in range(len(headers)):
        table += f'{headers[i]:<15}'
    table += '\n'
    for line in lines_info:
    #   # this looks bad, but it is to align the columns and adjust the number of decimals
        one_line_info = f'{line:<15}'
        one_line_info += f'{lines_info[line]["true_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["calib_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["area"]:<15.1f}'
        one_line_info += f'{lines_info[line]["max_counts"]:<15.1f}'
        one_line_info += f'{lines_info[line]["sigma"]:<15.4f}'
        one_line_info += f'{lines_info[line]["fwhm"]:<15.4f}'
        table += one_line_info + '\n'
    
    if give_return == True:
        return table
    else:
        print(table)

        

In [82]:
calibrate_lines(rounds=2, xray_lines='all_alpha')

Calibrating peak positions, round 1 of 2
Calibrating peak positions, round 2 of 2


In [83]:
lines_info = make_lines_info()
print_lines_info()

Line           True E [keV]   Calib. E [keV] Area [counts]  Max (fit)      Sigma [keV]    FWHM [eV]      
Al_Ka          1.4865         1.4865         91822.9        11214.1        0.0325         76.4774        
C_Ka           0.2774         0.2774         312027.1       54775.9        0.0226         53.1606        
Cu_Ka          8.0478         8.0478         68591.6        4317.1         0.0633         149.1712       
Mo_La          2.2932         2.2932         54820.9        5786.0         0.0377         88.6879        
Ni_Ka          7.4781         7.4781         342124.6       22259.8        0.0613         144.3184       
Ni_La          0.8511         0.8511         285763.3       41097.2        0.0277         65.2710        
O_Ka           0.5249         0.5249         217110.6       34085.8        0.0249         58.6924        
Si_Ka          1.7397         1.7397         25567.4        2983.3         0.0342         80.5096        



In [84]:
## line ratios

def peak_ratio(line1, line2):
    # give the K to L ratio of a line, e.g. 'Ga_Ka' to 'Ga_La'
    return gaussian_counts(line1) / gaussian_counts(line2)

def calculate_all_line_ratios():
    # calculate the K to L ratios for line_ratio_list
    line_ratios = {}
    for line_pair in line_ratio_list:
        pair_name = line_pair[0] + '/' + line_pair[1]
        line_ratios[pair_name] =  peak_ratio(line_pair[0], line_pair[0])
    return line_ratios

In [85]:
line_ratios = calculate_all_line_ratios()
line_ratios

{'Ni_Ka/Ni_La': 1.0}

In [86]:
## FWTM to FWHM

def fwtm_to_fwhm_line(line, plot_ratio=False):
    s_wo_bg = s.deepcopy()
    s_wo_bg.data = s.data - m[-1].function(x)*s.axes_manager[0].scale
    window = m[line].sigma.value * 4 # window around the line
    line_energy = m[line].centre.value
    s_line_window = s_wo_bg.isig[line_energy-window:line_energy+window]
    fwhm = s_line_window.estimate_peak_width(factor=0.5, parallel=False).data[0]
    fwtm = s_line_window.estimate_peak_width(factor=0.1, parallel=False).data[0]

    # TODO: Put this plotting in a separate function?
    if plot_ratio:
        plt.plot(s_line_window.axes_manager[0].axis, s_line_window.data, label=f'Signal without bg', marker='o')
        plt.axvline(line_energy-window, color='k', linestyle='--', label='window')
        plt.axvline(line_energy+window, color='k', linestyle='--')
        plt.axvline(theoretical_energy(line), color='r', linestyle='--', label='theoretical line center')
        plt.axvline(m[line].centre.value, color='g', linestyle='--', label='calibrated center')
        plt.plot([line_energy-fwhm/2, line_energy+fwhm/2], [lines_info[line]['max_counts']/2, lines_info[line]['max_counts']/2], label='fwhm')
        plt.plot([line_energy-fwtm/2, line_energy+fwtm/2], [lines_info[line]['max_counts']/10, lines_info[line]['max_counts']/10], label='fwtm')
        plt.title(f'{line} \n FWHM: {fwhm*1000:.2f} eV, FWTM: {fwtm*1000:.2f} eV, ratio: {fwtm/fwhm:.3f}')
        plt.legend()
        plt.show()
    
    return {'fwhm': fwhm*1000, 'fwtm': fwtm*1000, 'fwtm/fwhm': fwtm/fwhm}


def fwtm_to_fwhm_all():
    fwtm_to_fwhm = {}
    for line in fwtm_to_fwhm_lines:
        fwtm_to_fwhm[line] = fwtm_to_fwhm_line(line, plot_ratio=False)

    return fwtm_to_fwhm
        

fwtm_to_fwhm = fwtm_to_fwhm_all()
fwtm_to_fwhm

{'Ni_Ka': {'fwhm': 140.3527012161625,
  'fwtm': 257.71697327598633,
  'fwtm/fwhm': 1.8362095709085546}}

In [87]:
fwtm_to_fwhm_line('Ni_Ka', plot_ratio=True)

{'fwhm': 140.3527012161625,
 'fwtm': 257.71697327598633,
 'fwtm/fwhm': 1.8362095709085546}

In [88]:
# plt.plot(s.axes_manager[0].axis, s.data-m[-1].function(s.axes_manager[0].axis)*scale+100, label='spectrum', marker='o')
# # m_data = np.zeros_like(s.data)

# for comp in m:
#     if comp == m[-1]:
#         continue
#     # m_data += comp.function(s.axes_manager[0].axis)*scale

#     # if 'Sn_' in comp.name:
    
#     line_y = comp.height * scale
#     line_x = comp.centre.value
#     true_e  = hs.material.elements[comp.name.split('_')[0]].Atomic_properties.Xray_lines.as_dictionary()[comp.name.split('_')[1]]['energy (keV)']

#     plt.plot([line_x, line_x], [0, line_y+100])
#     plt.plot([true_e, true_e], [0, line_y+100], '--', color='k', alpha=0.7)

#     # plt.plot(s.axes_manager[0].axis, comp.function(s.axes_manager[0].axis)*scale+m[-1].function(s.axes_manager[0].axis)*scale, label=comp.name)
#     plt.plot(s.axes_manager[0].axis, comp.function(s.axes_manager[0].axis)*scale+100, label=comp.name)

#     plt.text(line_x, line_y+100, f'{comp.name}, {comp.centre.value:.3f}',rotation=90)

# # plt.plot(s.axes_manager[0].axis, m_data+150, label='model')


# plt.ylim(15,25e4)
# plt.yscale('log')

# # plt.legend()
# plt.show()

## Total counts and background counts

In [89]:
total_counts = s.data.sum()
# total_counts_m = m.signal.data.sum() # does not work, m.signal is just a view of s
print(f'total_counts: {total_counts:.1f}')

# counts in the background
background_counts = bg_range(x).sum()
print(f'background_counts: {background_counts:.1f}')

# counts in the spectrum without background
total_counts_wo_bg = total_counts - background_counts
print(f'total_counts_wo_bg: {total_counts_wo_bg:.1f}')

# ratio of total counts to total counts in background
print(f'total_counts/background_counts: {total_counts/background_counts:.1f}')

# ratio of total counts to total counts in spectrum without background
print(f'total_counts/total_counts_wo_bg: {total_counts/total_counts_wo_bg:.1f}')

# Fiori background at a certain peak
#TODO

total_counts: 2458125.0
background_counts: 703981.2
total_counts_wo_bg: 1754143.8
total_counts/background_counts: 3.5
total_counts/total_counts_wo_bg: 1.4


In [90]:
plt.close('all')

In [91]:
# setting the parameters which have temporary arrays

scale = scale_list[-1]
offset = offset_list[-1]
energy_resolution = energy_res_list[-1]

In [92]:
# TODO: discuss with Ton this Duane-Hunt method


# path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
# file = 'GaAs_10kV.emsa'
# s_dh = hs.load(path + '/' + file, signal_type='EDS_TEM')  # have to pretent it's a TEM signal
# s_dh.add_elements(elements)
# if zero_peak:
#     s_dh = s_dh.isig[zero_peak_end_index:]
# Vacc = s_dh.metadata.Acquisition_instrument.TEM.beam_energy
# x_max = s_dh.axes_manager[0].high_value  # highest x-axis value in keV, used in Duane-Hunt
# x = s_dh.axes_manager[0].axis  # x-axis in keV





# Duane-Hunt (where the the real E_0 is)
def calculate_duane_hunt(buffer_start=2, buffer_end=0.1, xaxis_buffer=0.5, dh_plot=False):
    if Vacc > x_max:
        print(f'Vacc={Vacc} > x_max={x_max}, Duane-Hunt not possible')
        return np.nan
    else:
        # doing a linear fit from Vacc-2 keV till Vacc-0.1 keV, i.e. only the bg
        dh_start = Vacc-buffer_start
        dh_end = Vacc-buffer_end
        s_end = s.isig[dh_start:dh_end] # slice with keV
        m_end = s_end.create_model(auto_background=False)
        m_end.add_polynomial_background(order=1)
        m_end.fit()

        # x-axis from dh_start-0.5 to dh_end+0.5, i.e. a buffer for plotting
        x_s_end = s.isig[dh_start-xaxis_buffer:dh_end+xaxis_buffer].axes_manager[0].axis

        # locate where the background is closest to zero
        bg_zero_index = np.argmin(np.abs(m_end[-1].function(x_s_end) * s.axes_manager[0].scale))

        dh_limit = x_s_end[bg_zero_index]

        print(f'bg_zero_index: {bg_zero_index}')
        print(f'Duane-Hunt limit: {dh_limit}')
        if dh_plot:
            plt.plot(x, s.data, label='spectrum', marker='o')
            plt.plot(x_s_end, m_end[-1].function(x_s_end)* s.axes_manager[0].scale, label='bg lin. fit')
            plt.plot(s_end.axes_manager[0].axis, s_end.data, marker='o', label=f'points in bg lin. fit')
            plt.axhline(0, color='k', linestyle='--')
            plt.axvline(dh_limit, color='r', linestyle='--', label=f'Duane-Hunt: {dh_limit:.2f} keV')
            plt.axvline(Vacc, color='r', linestyle='--', label=f'Vacc from instrument: {Vacc} keV', alpha=0.6)

            plt.legend()
            plt.title(f'{file}\nlin fit range kV: [{dh_start:.2f}, {dh_end:.2f}]')
            plot_buffer = 1.1
            plt.ylim(-5, m_end[-1].function(x_s_end).max() * s.axes_manager[0].scale*plot_buffer)
            plt.xlim(Vacc-buffer_start*plot_buffer, dh_limit + buffer_end)
            plt.show()
        return dh_limit




dh_limit = calculate_duane_hunt(dh_plot=True)

Vacc=30.0 > x_max=20.27, Duane-Hunt not possible


In [93]:
def print_output(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts, 
background_counts=background_counts, dh_limit=dh_limit, lines_info=lines_info, line_ratios=line_ratios, fwtm_to_fwhm=fwtm_to_fwhm):
    print(f"scale: {scale}")
    print(f"offset: {offset}")
    print(f"energy_resolution: {energy_resolution}")
    print(f"total_counts: {total_counts}")
    print(f"background_counts: {background_counts}")
    print(f"dh_limit: {dh_limit}")
    print(print_lines_info(give_return=True))
    print(f"line_ratios: {line_ratios}")
    print(f"fwtm_to_fwhm: {fwtm_to_fwhm}")

print_output()

scale: 0.009996773445464083
offset: 0.1348980721198133
energy_resolution: 129.92035855169993
total_counts: 2458125.0
background_counts: 703981.1630655867
dh_limit: nan
Line           True E [keV]   Calib. E [keV] Area [counts]  Max (fit)      Sigma [keV]    FWHM [eV]      
Al_Ka          1.4865         1.4865         91822.9        11214.1        0.0325         76.4774        
C_Ka           0.2774         0.2774         312027.1       54775.9        0.0226         53.1606        
Cu_Ka          8.0478         8.0478         68591.6        4317.1         0.0633         149.1712       
Mo_La          2.2932         2.2932         54820.9        5786.0         0.0377         88.6879        
Ni_Ka          7.4781         7.4781         342124.6       22259.8        0.0613         144.3184       
Ni_La          0.8511         0.8511         285763.3       41097.2        0.0277         65.2710        
O_Ka           0.5249         0.5249         217110.6       34085.8        0.0249         

In [94]:
# put output in a dictionary

def output_dict(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts,
background_counts=background_counts, lines_info=lines_info, line_ratios=line_ratios, fwtm_to_fwhm=fwtm_to_fwhm):
    output = {'scale': scale, 'offset': offset, 'energy_resolution': energy_resolution, 'total_counts': total_counts,
    'background_counts': background_counts, 'dh_limit':dh_limit, 'lines_info': lines_info, 'line_ratios': line_ratios, 'fwtm_to_fwhm': fwtm_to_fwhm}
    return output


output = output_dict()
output

{'scale': 0.009996773445464083,
 'offset': 0.1348980721198133,
 'energy_resolution': 129.92035855169993,
 'total_counts': 2458125.0,
 'background_counts': 703981.1630655867,
 'dh_limit': nan,
 'lines_info': {'Al_Ka': {'true_energy': 1.4865,
   'calib_energy': 1.4865,
   'area': 91822.85217193273,
   'max_counts': 11214.069251336288,
   'sigma': 0.03247698055799203,
   'fwhm': 76.47744482004005},
  'C_Ka': {'true_energy': 0.2774,
   'calib_energy': 0.2774,
   'area': 312027.0851628599,
   'max_counts': 54775.94811599539,
   'sigma': 0.02257522894254459,
   'fwhm': 53.16060163506684},
  'Cu_Ka': {'true_energy': 8.0478,
   'calib_energy': 8.0478,
   'area': 68591.62937514996,
   'max_counts': 4317.098862850585,
   'sigma': 0.0633471798492003,
   'fwhm': 149.1712089050775},
  'Mo_La': {'true_energy': 2.2932,
   'calib_energy': 2.2932,
   'area': 54820.901935666785,
   'max_counts': 5786.04504996606,
   'sigma': 0.03766229771440258,
   'fwhm': 88.68793359979851},
  'Ni_Ka': {'true_energy': 

In [95]:
plt.plot(x, m[-1].function(x) * s.axes_manager[0].scale)
plt.plot(x, s.data)

plt.show()

In [96]:
plt.close('all')

In [99]:
calculate_duane_hunt()

bg_zero_index: 202
Duane-Hunt limit: 9.521868337410586


9.521868337410586

In [101]:
Vacc

10.0

In [100]:
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 18.4
│       │       ├── energy_resolution_MnKa = 129.92035855169993
│       │       ├── live_time = 120.0
│       │       └── real_time = 137.728393555
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 30.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.3
│   │       ├── io_plugin = hyperspy.io_plugins.msa
│   │       ├── operation = load
│   │       └── timestamp = 2023-01-26T11:52:24.203112+01:00
│   ├── date = 2022-03-23
│   ├── original_filename = Spectrum 03.emsa
│   ├── time = 14:12:00
│   └── title = Spectrum 3
├── Sample
│   └── elements = ['Al', 'Au', 'C', 'Cu', 'F', 'Fe', 'Mg', 'Mo', 'Ni', 'O', 'Si', 'Sn']
└── Signal
    ├── quantity = X-rays (counts)
    └── signal_type = EDS_TEM